<a href="https://colab.research.google.com/github/TarunKannuchamy/ipl23_Scraping/blob/main/ipl23_Player_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Obtaining URLs**

In [ ]:
import requests
from bs4 import BeautifulSoup

url2 = 'https://www.espncricinfo.com/records/tournament/indian-premier-league-2023-15129?team=4343'
response2 = requests.get(url2)
soup = BeautifulSoup(response2.content, 'html.parser')
unwanted1 = soup.find('div', class_="ReactCollapse--collapse", style="height:auto;overflow:initial")
unwanted2 = unwanted1.find_next('div', class_="ReactCollapse--collapse", style="height:auto;overflow:initial")
link_classes = unwanted2.find_next('div', class_="ReactCollapse--collapse", style="height:auto;overflow:initial")
urls = []
for link_class in link_classes:
  a_tags = link_class.find_all('a', class_="ds-cursor-pointer ds-inline-flex ds-items-center ds-w-full ds-py-2 ds-text-tight-s ds-px-4 ds-bg-ui-fill ds-text-typo ds-flex ds-items-center ds-box-border hover:ds-bg-ui-fill-hover active:ds-bg-ui-fill-primary active:ds-text-raw-white focus:ds-border focus:ds-border-ui-stroke-primary focus:ds-outline-none")
  links = [f"https://www.espncricinfo.com{link.get('href')}" for link in a_tags if 'href' in link.attrs]
  urls.extend(links)


10


In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
# url = "https://www.espncricinfo.com/records/tournament/averages-batting-bowling-by-team/indian-premier-league-2023-15129?team=4343"
count = 1
for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  #Player Statistics
  values = soup.find_all("td", class_="ds-min-w-max ds-text-right")
  batting_stats = []
  bowling_stats = []
  index = 0
  for i,value in enumerate(values):
    if value.text == "2023-2023":
      index = i
      break
    else:
      batting_stats.append(value.text)
  batting_stats = batting_stats[13:-16]

  for value in values[index:]:
    bowling_stats.append(value.text)

  batting_matches = batting_stats[0::13]
  batting_innings = batting_stats[1::13]
  batting_not_outs = batting_stats[2::13]
  batting_runs = batting_stats[3::13]
  batting_balls_faced = batting_stats[4::13]
  batting_highest_score = batting_stats[5::13]
  batting_average = batting_stats[6::13]
  batting_strike_rate = batting_stats[7::13]
  batting_centuries = batting_stats[8::13]
  batting_fifties = batting_stats[9::13]
  batting_ducks = batting_stats[10::13]
  batting_fours = batting_stats[11::13]
  batting_sixes = batting_stats[12::13]

  bowling_span= bowling_stats[0::16]
  bowling_matches = bowling_stats[1::16]
  bowling_innings = bowling_stats[2::16]
  bowling_overs = bowling_stats[3::16]
  bowling_maidens = bowling_stats[4::16]
  bowling_runs = bowling_stats[5::16]
  bowling_wickets= bowling_stats[6::16]
  bowling_best = bowling_stats[7::16]
  bowling_average = bowling_stats[9::16]
  bowling_economy = bowling_stats[10::16]
  bowling_strike_rate = bowling_stats[11::16]
  bowling_five_wickets = bowling_stats[12::16]
  bowling_ten_wickets = bowling_stats[13::16]
  bowling_catches = bowling_stats[14::16]
  bowling_stumpings= bowling_stats[15::16]

  #Team Name
  team_name_html = soup.find("div", class_="ds-text-title-xs ds-font-bold")
  team_name = re.sub(r" bat.*", "", team_name_html.text)
  team_name_list = []
  n = int(len(bowling_stats)/16)
  for i in range(n):
    team_name_list.append(team_name)
  team_name_list

  #Player Name
  player_name_html = soup.find_all("span", class_="ds-text-tight-s ds-font-regular ds-text-typo-primary hover:ds-text-typo-primary-hover ds-block")
  bowler_name = []
  batter_name = []
  for i in player_name_html[:n]:
    batter_name.append(i.text)
  for i in player_name_html[n:]:
    bowler_name.append(i.text)

  #Appending to batting DataFrame
  df_batting = pd.DataFrame(zip(team_name_list, batter_name, batting_matches, batting_innings,
                                batting_not_outs, batting_runs, batting_balls_faced, batting_highest_score,
                                batting_average, batting_strike_rate,  batting_centuries, batting_fifties,
                                batting_ducks, batting_fours, batting_sixes),
                            columns = ['Team Name', 'Batter Name', 'Matches', 'Innings', "Not Outs",
                                      'Runs', 'Balls Faced', 'Highest Score', 'Average', 'Strike Rate',
                                      '100s', '50s', '0s', '4s', '6s'])

  #Appending to bowling DataFrame
  df_bowling = pd.DataFrame(zip(team_name_list, bowler_name, bowling_span, bowling_matches, bowling_innings, bowling_overs, bowling_maidens,
                                bowling_runs, bowling_wickets, bowling_best, bowling_average, bowling_economy, bowling_strike_rate,
                                bowling_five_wickets, bowling_ten_wickets, bowling_catches, bowling_stumpings),
                            columns = ['team_name_list', 'bowler_name', 'span', 'matches', 'innings', 'overs', 'maidens',
                                'runs', 'wickets', 'best', 'average', 'economy', 'strike_rate', 'five_wickets', 'ten_wickets',
                                'catches', 'stumpings'])

  #Writinf to CSVs
  csv_batting_file = 'ipl23_player_batting_stats.csv'
  csv_bowling_file = 'ipl23_player_bowling_stats.csv'
  df_batting.to_csv(csv_batting_file, mode='a', header=False, index=False)
  df_bowling.to_csv(csv_bowling_file, mode='a', header=False, index=False)
  print(f"Team - {count} - data saved successfully")
  count+=1

Team - 1 - data saved successfully
Team - 2 - data saved successfully
Team - 3 - data saved successfully
Team - 4 - data saved successfully
Team - 5 - data saved successfully
Team - 6 - data saved successfully
Team - 7 - data saved successfully
Team - 8 - data saved successfully
Team - 9 - data saved successfully
Team - 10 - data saved successfully
